In [1]:
from openai import OpenAI
import os

In [2]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [3]:
JOB_DESCRIPTION = """Job description of advertised OpenAI Technical Expert / Data Scientist position:
                TASKS:
                Development and implementation of Machine Learning / Artificial Intelligence models for applications, e.g. in the area of Natural Language Processing or Computer Vision, with a focus on OpenAI technologies to improve our services and processes
                Collaborate closely with other Data Scientists, subject matter experts, and external service providers
                Use case-based evaluation and consulting on the use of analytical methods and approaches (especially OpenAI technologies) to improve our processes and services
                Collaborate closely with our business units as an internal development partner, supporting them from the ideation phase to implementation
                QUALIFICATIONS:
                Master in computer science, mathematics, physics or a comparable qualification with a focus on Machine Learning / AI.
                At least three years of experience in implementing Machine Learning / AI models, including the use of OpenAI technologies
                In-depth expertise in the area of current Machine Learning / AI methods, especially Large Language Models
                Comprehensive knowledge of relevant programming languages, including Python
                Experience with cloud computing platforms, preferably Azure
                Knowledge of natural language processing and/or computer vision
                Strong analytical skills and the ability to solve complex problems
                Strong English communication skills are required, and basic knowledge of German is preferred.
                """

In [27]:
EVALUATION_PROMPT = f"""Job description: {JOB_DESCRIPTION}.
            As an AI interview assistant, your task is to evaluate the quality and depth of the candidate's responses and make a clear hiring decision for this job position.
            Consider the following:
            Does the candidate provide detailed answers that demonstrate their understanding and expertise?
            Can you find tangible examples in their responses that relate to the job description?
            Does the candidate elaborate on how they have used the necessary skills or experiences to overcome challenges or achieve results?
            Do the responses suggest the candidate has the ability to perform well in the role's complexities and challenges?
            If the candidate's responses are inadequate, vague, or don't clearly demonstrate the needed skills or experiences, they may not be a suitable match for the role. In such cases, tactfully communicate this by saying: "Thank you for your responses. However, based on the answers provided, it appears there may be a misalignment with the requirements of the role we're seeking to fill. At this time, we cannot extend an offer. We appreciate your time and effort and wish you the best in your future endeavors."
            If the responses indicate a strong fit for the role, then acknowledge the candidate's suitability by saying: "Thank you for your thoughtful responses. Based on your answers, it appears that your skills, experience, and understanding align well with the requirements of the role. We will be in touch with the next steps."
            """ 

In [30]:
def get_response_from_model(job_description=JOB_DESCRIPTION, nmb_of_questions=3):
    # Generate interview questions based on the job description
    prompt = f"Generate {nmb_of_questions} interview questions for a candidate applying for {job_description}."
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": "Question 1:"},
            {"role": "user", "content": "Question 2:"},
            {"role": "user", "content": "Question 3:"},
            # {"role": "user", "content": "Question 4:"},
            # {"role": "user", "content": "Question 5:"}
        ]
    )
    return response
 
def parse_model_response(response):  # TODO: check type openai.types.chat.chat_completion.ChatCompletion
    return [question for question in response.choices[0].message.content.split("\n") if question.strip()]  # TODO: use regex
    

def conduct_interview(questions, answers=[]):
    # Conduct the interview
    print("Welcome to the interview!")
    print("Please answer the following questions with a maximum of 20 words each:")
    for i, question in enumerate(questions, 1):
        print(f"Question {i}: {question}")
        answer = input("Your answer: ")
        answers.append(answer)
        print()
    print("Thank you for your answers. We will now evaluate your suitability for the position.")
    return answers

def make_hiring_decision(answers):
    # Make the hiring decision
    prompt = EVALUATION_PROMPT
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": " ".join(answers)}
        ]
    )
    return response

In [20]:
response = get_response_from_model()
questions = parse_model_response(response)
questions

['1. Can you provide an example of a significant project where you developed and implemented a Machine Learning model using OpenAI technologies in the area of Natural Language Processing or Computer Vision?',
 '2. How have you collaborated with cross-functional teams, including Data Scientists, subject matter experts, and external service providers, to drive the implementation of Machine Learning models in previous roles?',
 '3. Can you discuss a situation where you provided consulting services on the use of analytical methods, particularly OpenAI technologies, to improve processes and services within an organization? What were the outcomes of your recommendations?']

In [22]:
candidate_answers = conduct_interview(questions)

Welcome to the interview!
Please answer the following questions with a maximum of 20 words each:
Question 1: 1. Can you provide an example of a significant project where you developed and implemented a Machine Learning model using OpenAI technologies in the area of Natural Language Processing or Computer Vision?


Your answer:  In my previous role at [Company], I led a project focused on enhancing customer support efficiency using machine learning and OpenAI technologies. We leveraged OpenAI's GPT models to develop a chatbot capable of understanding and responding to customer queries in real-time.  The outcome of this project was twofold: Firstly, we saw a significant reduction in response times for customer inquiries, leading to increased customer satisfaction. Secondly, by automating routine tasks, our support team was able to focus on more complex issues, thereby improving overall service quality.  Moreover, we integrated sentiment analysis capabilities into the chatbot, allowing us to proactively address potential customer dissatisfaction before it escalated. This not only helped in retaining existing customers but also contributed to attracting new ones through positive word-of-mouth.  Overall, the project had a substantial impact on improving operational efficiency and customer experience,


Question 2: 2. How have you collaborated with cross-functional teams, including Data Scientists, subject matter experts, and external service providers, to drive the implementation of Machine Learning models in previous roles?


Your answer:  Staying current with the rapid advancements in Machine Learning and AI, particularly in the realm of Large Language Models, is crucial for maintaining relevance in this dynamic field. To ensure I stay abreast of the latest developments, I employ a multi-faceted approach.  Firstly, I regularly engage with online communities such as arXiv, Medium, and GitHub, where researchers and practitioners share their findings, code, and insights. This allows me to stay informed about cutting-edge research, emerging methodologies, and innovative applications in the field.  Additionally, I make it a point to attend conferences, workshops, and seminars, both in-person and virtually, to immerse myself in discussions about the latest trends and breakthroughs in Machine Learning and AI. These events provide valuable opportunities for networking, knowledge exchange, and exposure to diverse perspectives.  Furthermore, I actively participate in online courses, webinars, and tutorials offered b


Question 3: 3. Can you discuss a situation where you provided consulting services on the use of analytical methods, particularly OpenAI technologies, to improve processes and services within an organization? What were the outcomes of your recommendations?


Your answer:  n a previous role at [Company], I had the opportunity to collaborate with a multidisciplinary team consisting of Data Scientists, subject matter experts, and external service providers on a project aimed at developing a predictive maintenance model for industrial machinery.  Effective communication and coordination were essential for the success of this project, given the diverse expertise and perspectives involved. To ensure seamless collaboration, I implemented several strategies:  Firstly, I facilitated regular team meetings where we discussed project objectives, progress updates, and any challenges encountered. These meetings served as forums for brainstorming ideas, sharing insights, and aligning on priorities.  Secondly, I established clear roles and responsibilities for each team member, ensuring everyone understood their contributions and commitments to the project. This clarity helped minimize duplication of efforts and maximized productivity.  Thirdly, I leverag


Thank you for your answers. We will now evaluate your suitability for the position.


In [23]:
candidate_answers

['In my previous role at [Company], I led a project focused on enhancing customer support efficiency using machine learning and OpenAI technologies. We leveraged OpenAI\'s GPT models to develop a chatbot capable of understanding and responding to customer queries in real-time.  The outcome of this project was twofold: Firstly, we saw a significant reduction in response times for customer inquiries, leading to increased customer satisfaction. Secondly, by automating routine tasks, our support team was able to focus on more complex issues, thereby improving overall service quality.  Moreover, we integrated sentiment analysis capabilities into the chatbot, allowing us to proactively address potential customer dissatisfaction before it escalated. This not only helped in retaining existing customers but also contributed to attracting new ones through positive word-of-mouth.  Overall, the project had a substantial impact on improving operational efficiency and customer experience, underscori

In [31]:
result = make_hiring_decision(candidate_answers)
result

ChatCompletion(id='chatcmpl-9N4LVPlVC9aIlj2Fgd90swBqahiMJ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Thank you for your detailed responses that showcase your experience and expertise in the areas of Machine Learning, AI, and OpenAI technologies. Your project at [Company] focused on enhancing customer support efficiency using machine learning and OpenAI technologies, particularly leveraging OpenAI's GPT models to develop a chatbot. The outcomes you achieved, such as reduced response times, improved customer satisfaction, and automation of routine tasks leading to better service quality, align well with the responsibilities outlined for the OpenAI Technical Expert / Data Scientist position.\n\nYour proactive approach to staying updated with the latest advancements in Machine Learning and AI, including attending conferences, engaging in online communities, and participating in online educational resources, demonstrates a commitme